In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core import tools
from langchain.tools import tool
from IPython.display import Markdown


load_dotenv("./.env")

llm = ChatOpenAI(model="gpt-4o")



In [10]:
@tool 
def write_email(to: str, subject:str, content: str) -> str:
    """Write and send email"""
    return f"Email sent to {to} with subject {subject} and content : {content}"

model_with_tools = llm.bind_tools( [write_email] , parallel_tool_calls=False)
output = model_with_tools.invoke("Envia un email a manuonda@gmail.com diciendo que es un crack")

print(output.tool_calls[0])
print(f"Name tool : {output.tool_calls[0]['name']}")

type(output)
args = output.tool_calls[0]['args']
print(f"Args : {args}")

#call the tool
result = write_email.invoke(args)
Markdown(result)


{'name': 'write_email', 'args': {'to': 'manuonda@gmail.com', 'subject': 'Eres un crack!', 'content': 'Hola,\n\nSolo quería tomarme un momento para decir que eres un auténtico crack. Sigue con el buen trabajo y esa actitud positiva.\n\n¡Saludos!'}, 'id': 'call_AyBDvC9NsDadiiBgMD5XBqdd', 'type': 'tool_call'}
Name tool : write_email
Args : {'to': 'manuonda@gmail.com', 'subject': 'Eres un crack!', 'content': 'Hola,\n\nSolo quería tomarme un momento para decir que eres un auténtico crack. Sigue con el buen trabajo y esa actitud positiva.\n\n¡Saludos!'}


Email sent to manuonda@gmail.com with subject Eres un crack! and content : Hola,

Solo quería tomarme un momento para decir que eres un auténtico crack. Sigue con el buen trabajo y esa actitud positiva.

¡Saludos!

In [13]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)




def write_email_node(state: StateSchema) -> StateSchema:
    # Imperative code that processes the request
    output = model_with_tools.invoke(state["request"])
    print(output)
    if output.tool_calls:
      args = output.tool_calls[0]['args']
      email = write_email.invoke(args)
      return {"email": email}
    else:
        return {"email": output.content}



workflow = StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

app.invoke({"request": "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"})#


content='Sure, I can help draft an email for you. Could you please provide me with some details about the meeting or any specific points you would like to include in the response?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 66, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-Bqq4szeQCmVtbBaL9DyEuudY0pwPn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--aa7e1f69-3e93-4483-94c1-647792bf13fa-0' usage_metadata={'input_tokens': 66, 'output_tokens': 36, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


{'request': "Draft a response to my boss (boss@company.ai) about tomorrow's meeting",
 'email': 'Sure, I can help draft an email for you. Could you please provide me with some details about the meeting or any specific points you would like to include in the response?'}

In [ ]:
#should_continue 
from typing import Literal
from langgraph.graph import MessagesState
from email_assistant.utils import show_graph 

def call_llm(state: MessagesState) -> MessagesState:
    """ Run LLM """
    output = model_with_tools.invoke(state['messages'])
    return {"messages": [output]}

def run_tool(state: MessagesState):
    """Performs the tool call"""
    result = []
    for tool_call in state["messages"][-1].tools_calls:
        observation = write_email.invoke(tool_call["args"])
        result.append({"role":"tool", "content":observation, "tool_call_id": tool_call["id"]})
    return {"messages": result}

def should_continue(state:MessagesState)->Literal["run_tool","__end__"]:
    """Route to tool handler, or end if Done tool called"""
    #GEt the las message
    messages= state["messages"]
    last_message= messages[-1]
    if last_message.tool_calls:
        return "tool"
      
        
